In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
pos = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_POS_CASH_balance.csv')

In [18]:
pos

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,1803195,-31,48.0,45.0,Active,0,0,185279
1,1803195,-17,48.0,31.0,Active,0,0,185279
2,1803195,-21,48.0,35.0,Active,0,0,185279
3,1803195,-8,48.0,21.0,Active,0,0,185279
4,1803195,-4,48.0,17.0,Active,0,0,185279
...,...,...,...,...,...,...,...,...
8543370,2340627,-2,NaN,NaN,Signed,0,0,284649
8543371,1011796,-2,6.0,6.0,Active,0,0,69172
8543372,1298851,-1,12.0,12.0,Active,0,0,151122
8543373,1550592,-1,1.0,0.0,Completed,0,0,56549


In [19]:
print('Columns that contain more than 90% missing values:')
more90_cols = pos.loc[:,(pos.isnull().sum(axis = 0) / pos.shape[0]) >= 0.9].columns
more90_cols

Columns that contain more than 90% missing values:


Index([], dtype='object')

NO COLUMNS HAVE NULL VALUES > 90% -> SO WE'RE KEEPING THE DATASET AS IT IS FOR NOW

## Duplicates Check

In [20]:
pos.duplicated().sum()

0

# Features Selection and Engineer

In [21]:
# tao cot de kiem tra so lan bi qua han khi thanh toan khoan vay
def check_over_due(x):
    if x['SK_DPD'] > 0 and x['SK_DPD_DEF'] > 0:
        return 1
    else:
        return 0

pos['over_due'] =  pos.apply(check_over_due,axis= 1)

* Đây chính là thông tin cập nhật hàng tháng của các khoản vay POS, CASH theo từng tháng cho nên ta lấy thông tin cập nhật mới nhất theo từng tháng của từng SK_ID_PREV


In [22]:
pos_filter = pos.sort_values(by=['MONTHS_BALANCE']).groupby(["SK_ID_CURR", "SK_ID_PREV"]).last().reset_index()

# Tao them 1 cot dem so lan ma cac khoan CASH, POS duoc thanh toan hoan tat
pos_filter['Payment_Counts'] = pos.sort_values(by=['MONTHS_BALANCE']).groupby(["SK_ID_CURR", "SK_ID_PREV"])['MONTHS_BALANCE']\
    .agg('count').reset_index()['MONTHS_BALANCE']

# Tao them mot cot de kiem tra xem khoan vay do da duoc tra hoan tat hay chua (0 la chua, 1 la co)
pos_filter["Completed_STATUS"] = pos_filter["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x== "Completed" else 0)
pos_filter = pos_filter.drop(columns='NAME_CONTRACT_STATUS',axis = 1)


In [23]:
# tao them 1 cot dem xem trong 1 khoan vay ho nop tien muon bao nhieu lan

sk_dpd_count_overdue = pos[pos['SK_DPD'] > 0].sort_values(by=['MONTHS_BALANCE'])\
    .groupby(["SK_ID_CURR", "SK_ID_PREV"])['SK_DPD'].agg('count').reset_index()\
        .rename(columns={'SK_DPD':'Count_times_over_due'})

pos_filter = pd.merge(pos_filter, sk_dpd_count_overdue , on=['SK_ID_CURR', 'SK_ID_PREV'], how='left')
# cot nao ma NAn thi fill = 0
pos_filter['Count_times_over_due'] = pos_filter['Count_times_over_due'].fillna(0)
pos_filter = pos_filter.drop(columns='over_due')

In [24]:
pos_filter

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,Payment_Counts,Completed_STATUS,Count_times_over_due
0,0,1231213,-1,12.0,12.0,0,0,1,0,0.0
1,0,1716062,-5,2.0,0.0,0,0,4,1,0.0
2,0,1891354,-1,3.0,0.0,0,0,4,1,0.0
3,0,2665996,-10,5.0,0.0,0,0,6,1,0.0
4,1,1847729,-43,10.0,0.0,0,0,11,1,0.0
...,...,...,...,...,...,...,...,...,...,...
800332,307508,1956312,-1,10.0,2.0,0,0,9,0,0.0
800333,307508,1978855,-64,12.0,1.0,0,0,12,0,0.0
800334,307509,1153961,-43,8.0,0.0,0,0,9,1,0.0
800335,307509,2384616,-25,6.0,0.0,0,0,7,1,0.0


#### Note

+ Because SK_ID_CURR in this table is duplicated, so we need to group by all of the data so that we can join with the application train to prevent wrong merge table


In [25]:
final_pos_cash = pos_filter.groupby(['SK_ID_CURR']).agg({
    'MONTHS_BALANCE':['sum','mean','max','min'],
    'CNT_INSTALMENT':['sum','mean','max','min'],
    'CNT_INSTALMENT_FUTURE':['sum','mean','max','min'],
    'Payment_Counts':['sum','mean','max','min'],
    'Completed_STATUS': 'sum',
    'Count_times_over_due': ['sum','mean','max','min']
})
final_pos_cash.columns = ['_'.join(col).strip().upper() for col in final_pos_cash.columns.values]
final_pos_cash = final_pos_cash.reset_index()

In [26]:
final_pos_cash

,SK_ID_CURR,MONTHS_BALANCE_SUM,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_MIN,CNT_INSTALMENT_SUM,CNT_INSTALMENT_MEAN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_FUTURE_SUM,...,CNT_INSTALMENT_FUTURE_MIN,PAYMENT_COUNTS_SUM,PAYMENT_COUNTS_MEAN,PAYMENT_COUNTS_MAX,PAYMENT_COUNTS_MIN,COMPLETED_STATUS_SUM,COUNT_TIMES_OVER_DUE_SUM,COUNT_TIMES_OVER_DUE_MEAN,COUNT_TIMES_OVER_DUE_MAX,COUNT_TIMES_OVER_DUE_MIN
0,0,-17,-4.250000,-1,-10,22.0,5.500000,12.0,2.0,12.0,...,0.0,15,3.750000,6,1,3,0.0,0.0,0.0,0.0
1,1,-55,-27.500000,-12,-43,21.0,10.500000,11.0,10.0,0.0,...,0.0,23,11.500000,12,11,2,2.0,1.0,2.0,0.0
2,2,-222,-55.500000,-23,-73,30.0,7.500000,12.0,4.0,1.0,...,0.0,34,8.500000,13,5,3,0.0,0.0,0.0,0.0
3,3,-37,-12.333333,-5,-22,11.0,3.666667,5.0,2.0,0.0,...,0.0,14,4.666667,6,3,3,0.0,0.0,0.0,0.0
4,4,-73,-73.000000,-73,-73,6.0,6.000000,6.0,6.0,0.0,...,0.0,7,7.000000,7,7,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289439,307505,-98,-49.000000,-40,-58,18.0,9.000000,10.0,8.0,0.0,...,0.0,20,10.000000,11,9,2,0.0,0.0,0.0,0.0
289440,307506,-109,-27.250000,-2,-75,42.0,10.500000,12.0,6.0,0.0,...,0.0,47,11.750000,13,8,4,4.0,1.0,4.0,0.0
289441,307508,-92,-23.000000,-1,-64,44.0,11.000000,12.0,10.0,3.0,...,0.0,46,11.500000,14,9,2,0.0,0.0,0.0,0.0
289442,307509,-68,-34.000000,-25,-43,14.0,7.000000,8.0,6.0,0.0,...,0.0,16,8.000000,9,7,2,0.0,0.0,0.0,0.0


# Fill null

In [27]:
final_pos_cash.isnull().sum() / final_pos_cash.shape[0]

SK_ID_CURR                    0.000000
MONTHS_BALANCE_SUM            0.000000
MONTHS_BALANCE_MEAN           0.000000
MONTHS_BALANCE_MAX            0.000000
MONTHS_BALANCE_MIN            0.000000
CNT_INSTALMENT_SUM            0.000000
CNT_INSTALMENT_MEAN           0.000083
CNT_INSTALMENT_MAX            0.000083
CNT_INSTALMENT_MIN            0.000083
CNT_INSTALMENT_FUTURE_SUM     0.000000
CNT_INSTALMENT_FUTURE_MEAN    0.000083
CNT_INSTALMENT_FUTURE_MAX     0.000083
CNT_INSTALMENT_FUTURE_MIN     0.000083
PAYMENT_COUNTS_SUM            0.000000
PAYMENT_COUNTS_MEAN           0.000000
PAYMENT_COUNTS_MAX            0.000000
PAYMENT_COUNTS_MIN            0.000000
COMPLETED_STATUS_SUM          0.000000
COUNT_TIMES_OVER_DUE_SUM      0.000000
COUNT_TIMES_OVER_DUE_MEAN     0.000000
COUNT_TIMES_OVER_DUE_MAX      0.000000
COUNT_TIMES_OVER_DUE_MIN      0.000000
dtype: float64

* the %null is low -> we can drop it all


In [28]:
final_pos_cash = final_pos_cash.dropna()

In [29]:
final_pos_cash = final_pos_cash.set_index('SK_ID_CURR')
final_pos_cash

,MONTHS_BALANCE_SUM,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_MIN,CNT_INSTALMENT_SUM,CNT_INSTALMENT_MEAN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_FUTURE_SUM,CNT_INSTALMENT_FUTURE_MEAN,...,CNT_INSTALMENT_FUTURE_MIN,PAYMENT_COUNTS_SUM,PAYMENT_COUNTS_MEAN,PAYMENT_COUNTS_MAX,PAYMENT_COUNTS_MIN,COMPLETED_STATUS_SUM,COUNT_TIMES_OVER_DUE_SUM,COUNT_TIMES_OVER_DUE_MEAN,COUNT_TIMES_OVER_DUE_MAX,COUNT_TIMES_OVER_DUE_MIN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-17,-4.250000,-1,-10,22.0,5.500000,12.0,2.0,12.0,3.00,...,0.0,15,3.750000,6,1,3,0.0,0.0,0.0,0.0
1,-55,-27.500000,-12,-43,21.0,10.500000,11.0,10.0,0.0,0.00,...,0.0,23,11.500000,12,11,2,2.0,1.0,2.0,0.0
2,-222,-55.500000,-23,-73,30.0,7.500000,12.0,4.0,1.0,0.25,...,0.0,34,8.500000,13,5,3,0.0,0.0,0.0,0.0
3,-37,-12.333333,-5,-22,11.0,3.666667,5.0,2.0,0.0,0.00,...,0.0,14,4.666667,6,3,3,0.0,0.0,0.0,0.0
4,-73,-73.000000,-73,-73,6.0,6.000000,6.0,6.0,0.0,0.00,...,0.0,7,7.000000,7,7,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307505,-98,-49.000000,-40,-58,18.0,9.000000,10.0,8.0,0.0,0.00,...,0.0,20,10.000000,11,9,2,0.0,0.0,0.0,0.0
307506,-109,-27.250000,-2,-75,42.0,10.500000,12.0,6.0,0.0,0.00,...,0.0,47,11.750000,13,8,4,4.0,1.0,4.0,0.0
307508,-92,-23.000000,-1,-64,44.0,11.000000,12.0,10.0,3.0,0.75,...,0.0,46,11.500000,14,9,2,0.0,0.0,0.0,0.0


In [30]:
final_pos_cash.to_csv("D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\pos_cash.csv")